In [140]:
import numpy as np
geometry = np.load('/Users/nocquet/projets/2020/afterslip_pedernales/geometry/n8_geometry.npy')

class topology_trimesh:
  def __init__(self):
    pass

def make_topology_trimesh( geometry ):

  # tolerance
  tol = 1.E-6

  # import
  import numpy as np
  # topology
  topology = topology_trimesh()

  # get distance matrix
  np_coor_vertices_1 = geometry[:,12:14]
  np_coor_vertices_2 = geometry[:,15:17]
  np_coor_vertices_3 = geometry[:,18:20]
  np_coor_vertices = np.vstack((np_coor_vertices_1,np_coor_vertices_2))
  np_coor_vertices = np.vstack((np_coor_vertices,np_coor_vertices_3))

  from scipy.spatial import distance_matrix
  Dm = distance_matrix(np_coor_vertices,np_coor_vertices)

  # convert distance matrix to index

  np_cell_vertices = np.zeros(( geometry.shape[0],3),dtype=int) * np.nan

  idx = 0
  lvertex_coor = []

  for i in np.arange(Dm.shape[0]):
    idx_cell = np.remainder(i,geometry.shape[0])
    idx_vertex_in_cell = np.floor_divide(i , geometry.shape[0])
    lindex=np.where(Dm[i,:] < tol)[0]
    tindex = np.array([ np.remainder(lindex , geometry.shape[0] ), np.floor_divide(lindex , geometry.shape[0]) ]).T
    if [idx_cell,idx_vertex_in_cell] != tindex[0,:].tolist():
        np_cell_vertices[tindex[:,0],tindex[:,1]] = np_cell_vertices[tindex[0,0],tindex[0,1]]
    else:
        np_cell_vertices[tindex[:,0],tindex[:,1]] = idx
        lvertex_coor.append([np_coor_vertices[i,:]])
        idx=idx+1

  topology.cell_vertex_idx = np_cell_vertices.astype(np.int64)
  topology.vertex_coor = np.squeeze( np.array(lvertex_coor) )

  # vertex_cell_idx: for a vertex idx, gives a list of cells having this vertex
  topology.vertex_cell_idx = {}

  for i in np.arange( topology.vertex_coor.shape[0] ):
      topology.vertex_cell_idx[i] =  np.argwhere( topology.cell_vertex_idx == i )

  # neighbours
  topology.cell_neighbours_edge = {}
  topology.cell_neighbours_vertex = {}
  for i in np.arange( topology.cell_vertex_idx.shape[0] ):
      topology.cell_neighbours_edge[i]   = []
      topology.cell_neighbours_vertex[i] = []

      list_cell_idx = []
      list_cell_idx = topology.vertex_cell_idx[ topology.cell_vertex_idx[i,0]][:,0].tolist()
      list_cell_idx +=  topology.vertex_cell_idx[ topology.cell_vertex_idx[i,1]][:,0].tolist()
      list_cell_idx += topology.vertex_cell_idx[ topology.cell_vertex_idx[i,2]][:,0].tolist()
      list_cell_idx = [item for item in list_cell_idx if item != i]
      list_cell_idx = sorted( list_cell_idx )

      for idx_neighbour in list(set(list_cell_idx)):
          if list_cell_idx.count(idx_neighbour) ==2:
              topology.cell_neighbours_edge[i].append(idx_neighbour)
          if list_cell_idx.count(idx_neighbour) ==1:
              topology.cell_neighbours_vertex[i].append(idx_neighbour)


  return topology

topology = make_topology_trimesh( geometry )

In [136]:
def make_discrete_laplace_trimesh( geometry ):
    # get topology
    topology = make_topology_trimesh( geometry )
    # compute the discrete_laplace operator (DLO)
    # create a sparse matrix
    from scipy.sparse import lil_matrix
    DLO = lil_matrix((geometry.shape[0], geometry.shape[0]))
    # loop on cells
    for i in np.arange( DLO.shape[0] ):
        DLO[i,topology.cell_neighbours_edge[i]]   = -1./6
        DLO[i,topology.cell_neighbours_vertex[i]] = -1./18
        DLO[i,i] = - DLO[i,:].sum(axis=1)

    print(DLO)

In [156]:
from scipy.sparse import lil_matrix
A = lil_matrix((5, 5))
A[1,1] = 1
A[2,3] = 2
B = np.zeros((5,5))
print(B+A)
B[0,[2,4]] = 8
print(B)
print(A)
print(A@A)
print('A')
print(A)
print('A.T*A from sparse')
print((A.T).dot(A).toarray())
print('A.T*A from dense')
AD = A.toarray()
np.dot(AD.T,AD)

[[0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 8. 0. 8.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
  (1, 1)	1.0
  (2, 3)	2.0
  (1, 1)	1.0
A
  (1, 1)	1.0
  (2, 3)	2.0
A.T*A from sparse
[[0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 4. 0.]
 [0. 0. 0. 0. 0.]]
A.T*A from dense


array([[0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 4., 0.],
       [0., 0., 0., 0., 0.]])

In [141]:
make_discrete_laplace_trimesh( geometry )

  (0, 0)	0.5555555555555556
  (0, 1)	-0.05555555555555555
  (0, 8)	-0.16666666666666666
  (0, 9)	-0.16666666666666666
  (0, 16)	-0.05555555555555555
  (0, 17)	-0.05555555555555555
  (0, 25)	-0.05555555555555555
  (1, 0)	-0.05555555555555555
  (1, 1)	0.6111111111111112
  (1, 2)	-0.05555555555555555
  (1, 9)	-0.16666666666666666
  (1, 10)	-0.16666666666666666
  (1, 17)	-0.05555555555555555
  (1, 18)	-0.05555555555555555
  (1, 26)	-0.05555555555555555
  (2, 1)	-0.05555555555555555
  (2, 2)	0.6111111111111112
  (2, 3)	-0.05555555555555555
  (2, 10)	-0.16666666666666666
  (2, 11)	-0.16666666666666666
  (2, 18)	-0.05555555555555555
  (2, 19)	-0.05555555555555555
  (2, 27)	-0.05555555555555555
  (3, 2)	-0.05555555555555555
  (3, 3)	0.6111111111111112
  (3, 4)	-0.05555555555555555
  (3, 11)	-0.16666666666666666
  (3, 12)	-0.16666666666666666
  (3, 19)	-0.05555555555555555
  (3, 20)	-0.05555555555555555
  (3, 28)	-0.05555555555555555
  (4, 3)	-0.05555555555555555
  (4, 4)	0.6111111111111112
  (

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])